## Snapshot and GraphQL

In [1]:
import requests
import pandas as pd
import numpy as np

Snapshot Endpoints

https://docs.snapshot.org/

In [2]:
ENDPOINT_DEMO = 'https://testnet.snapshot.org/graphql'

ENDPOINT_PRO = 'https://hub.snapshot.org/graphql'


ENDPOINT = ENDPOINT_PRO


We can make GraphQL queries with standard POST requests.

In [3]:
# https://datagy.io/python-requests-authentication/

def api(query, params=None):

    response = requests.post(ENDPOINT,
                            headers={                      
                                'accept': 'application/json'
                            },
                            params={
                                'query': query
                            })

    print(response)
    return response.json()['data']


In [4]:
query = """
query {
  space(id: "yam.eth") {
    id
    name
    about
    network
    symbol
    members
  }
}
"""

res = api(query)

<Response [200]>


In [5]:
res


{'space': {'id': 'yam.eth',
  'name': 'Yam',
  'about': 'Only delegated YAM may be used to vote on proposals. You can delegate to yourself or another address here: yam.finance/#/delegate',
  'network': '1',
  'symbol': 'YAM',
  'members': ['0x683A78bA1f6b25E29fbBC9Cd1BFA29A51520De84',
   '0x9Ebc8AD4011C7f559743Eb25705CCF5A9B58D0bc',
   '0xC3edCBe0F93a6258c3933e86fFaA3bcF12F8D695',
   '0xbdac5657eDd13F47C3DD924eAa36Cf1Ec49672cc',
   '0xEC3281124d4c2FCA8A88e3076C1E7749CfEcb7F2']}}

In [6]:
query = """
query {
  spaces(
    first: 20,
    skip: 0,
    orderBy: "created",
    orderDirection: asc
  ) {
    id
    name
    about
    network
    symbol
    strategies {
      name
      params
    }
    admins
    members
    filters {
      minScore
      onlyMembers
    }
    plugins
  }
}
"""

res = api(query)

<Response [200]>


In [7]:

res

{'spaces': [{'id': 'bonustrack.eth',
   'name': 'Hi',
   'about': '',
   'network': '1',
   'symbol': 'TICKET',
   'strategies': [{'name': 'ticket', 'params': {'symbol': 'TICKET'}}],
   'admins': [],
   'members': ['0x24A12Fa313F57aF541d447c594072A992c605DCf',
    '0x8BBE4Ac64246d600BC2889ef5d83809D138F03DF',
    '0xeF8305E140ac520225DAf050e2f71d5fBcC543e7',
    '0x107174D70b33523E83711EA4C3F4229cdB8A7Cb1',
    '0x0b21Dbc34A953a35dFA629ad31e1cEE3dE0433Fb',
    '0x40FDAa1bff3BB87bC70955Be6657edBFdBeb122e',
    '0xcc6A949DB9b26a7173648d50Cf7C55e800E6585B',
    '0xC1C8ac723Fd9Db3db6E947dF42aaEdd28058A31D'],
   'filters': {'minScore': 0, 'onlyMembers': False},
   'plugins': {'safeSnap': {'address': '0xeF8305E140ac520225DAf050e2f71d5fBcC543e7'}}},
  {'id': 'fabien.eth',
   'name': 'Fabien',
   'about': 'This is nothing more than a test space.',
   'network': '1',
   'symbol': 'POINT',
   'strategies': [{'name': 'ticket',
     'params': {'value': 100, 'symbol': 'POINT'}}],
   'admins': ['0xe

What if want to fetch all spaces? It's a bit cumbersome with standard POST requests, but we can use the [gql](https://pypi.org/project/gql/) client. 

#### GQL Client

In [9]:
!pip install gql
!pip install aiohttp 
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 KB 1.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 KB 3.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.7 MB/s eta 0:00:00a 0:00:01


In [10]:
transport = AIOHTTPTransport(url=ENDPOINT)
client = Client(
    transport=transport
    #fetch_schema_from_transport=True
)


Let's test a simple query.
First, we need build it, when we execute it.

_Note:_ gql will throw an error if the query is malformed.

In [11]:
space_query = gql(
"""
query {
space(id: "yam.eth") {
    id
    name
    about
    network
    symbol
    members
    }
}
""")

In [12]:
## Error in Jupyter-like environments.
result = client.execute(space_query)
print(result)

AssertionError: Cannot run client.execute(query) if an asyncio loop is running. Use 'await client.execute_async(query)' instead.

In [13]:
result = await client.execute_async(space_query)
print(result)

{'space': {'id': 'yam.eth', 'name': 'Yam', 'about': 'Only delegated YAM may be used to vote on proposals. You can delegate to yourself or another address here: yam.finance/#/delegate', 'network': '1', 'symbol': 'YAM', 'members': ['0x683A78bA1f6b25E29fbBC9Cd1BFA29A51520De84', '0x9Ebc8AD4011C7f559743Eb25705CCF5A9B58D0bc', '0xC3edCBe0F93a6258c3933e86fFaA3bcF12F8D695', '0xbdac5657eDd13F47C3DD924eAa36Cf1Ec49672cc', '0xEC3281124d4c2FCA8A88e3076C1E7749CfEcb7F2']}}


#### GraphQL Variables

Now we want to fetch _all_ spaces.

We need to use a special feature of GraphQL, that is the ability to pass **variables** into queries.

Variables are identified by the dollar sign `$` and must indicate their type (e.g., `Int`). 

If a variable is mandatory, it is followed by an exclamation mark.

In [14]:

spaces_query = gql("""
  query ($first: Int!, $skip: Int!) {
    spaces(
      first: $first,
      skip: $skip,
      orderBy: "created",
      orderDirection: asc
    ) 
    {
      id
      name
      about
      network
      symbol
      strategies {
        name
        params
      }
      admins
      members
      filters {
        minScore
        onlyMembers
      }
      plugins
    }
  }
""")


In [15]:
## spaces_tmp is a temp variable we can use to store partial computations
## in case an error (e.g., a timeout) occurs
spaces_tmp = []

In [16]:

## Fetch'em all!
spaces = spaces_tmp
first = 1000
skip = len(spaces)
fetch = True
while fetch:
    vars = {"first": first, "skip": skip}
    res = await client.execute_async(spaces_query, variable_values=vars)
    # print(type(res))
    # print(res)
    
    if not res['spaces']:
        print('**I am done fetching!**')
        fetch = False
    else:
        spaces.extend(res['spaces'])
        print(len(spaces))
        skip += first# fetch = False


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
10956
**I am done fetching!**


In [17]:
len(spaces)


10956

Let's put everything in a Pandas data frame.

In [20]:
df = pd.DataFrame(spaces)

df['id'] = df['id'].astype('string')
df['name'] = df['name'].astype('string')
df['about'] = df['about'].astype('string')
# df['logo'] = df['logo'].astype('string')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10956 entries, 0 to 10955
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          10956 non-null  string
 1   name        10956 non-null  string
 2   about       10956 non-null  string
 3   network     10956 non-null  object
 4   symbol      10956 non-null  object
 5   strategies  10956 non-null  object
 6   admins      10956 non-null  object
 7   members     10956 non-null  object
 8   filters     10956 non-null  object
 9   plugins     10956 non-null  object
dtypes: object(7), string(3)
memory usage: 856.1+ KB


In [24]:
df.to_json("data/daos_snapshot.json", orient="records")

## Visually explore the saved file.
# Is it a mess? Prettify it with VS Code auto-formatter. 
# Don't you remember the shorcut? Check our python_warmup lecture.
 

In [25]:
df.head()


,id,name,about,network,symbol,strategies,admins,members,filters,plugins
0,bonustrack.eth,Hi,,1,TICKET,"[{'name': 'ticket', 'params': {'symbol': 'TICK...",[],"[0x24A12Fa313F57aF541d447c594072A992c605DCf, 0...","{'minScore': 0, 'onlyMembers': False}",{'safeSnap': {'address': '0xeF8305E140ac520225...
1,fabien.eth,Fabien,This is nothing more than a test space.,1,POINT,"[{'name': 'ticket', 'params': {'value': 100, '...","[0xeF8305E140ac520225DAf050e2f71d5fBcC543e7, 0...",[0xeF8305E140ac520225DAf050e2f71d5fBcC543e7],"{'minScore': 1, 'onlyMembers': False}","{'safeSnap': {'safes': [{'network': '4', 'real..."
2,ichi.eth,ichi.farm,,1,ICHIPOWAH,"[{'name': 'erc20-balance-of', 'params': {'symb...",[],[0x873902a3c0731EcC1F2075435FE035ACCeEd5459],"{'minScore': 0, 'onlyMembers': False}",{}
3,loyalfinance.eth,loyal.finance,,1,rLYL,"[{'name': 'erc20-balance-of', 'params': {'symb...",[],[],"{'minScore': 0, 'onlyMembers': False}",{}
4,damflux.eth,FLUX,,1,FLUX,"[{'name': 'erc20-balance-of', 'params': {'symb...",[],[],"{'minScore': 0, 'onlyMembers': False}",{}


Now let's create a function to make our life easier.

The function `gql_all` will fetch all paginated responses from an endpoint until it returns an empty response.

_Notice:_ the _async_ keyword and the _try/except_ statement

In [ ]:
async def gql_all(query, field, first=1000, skip=0, initial_list=None, 
                  counter = True):

    out = []
    
    if initial_list:
        out = initial_list
        skip = len(out)

    fetch = True
    try:
        while fetch:
            vars = {"first": first, "skip": skip}
            res = await client.execute_async(query, variable_values=vars)
            # print(type(res))
            # print(res)
            
            if not res[field]:
                print('**I am done fetching!**')
                fetch = False
            else:
                out.extend(res[field])
                skip += first# fetch = False
                if counter: print(len(out))
    except:
        print("**An error occurred, exiting early.**")
    
    return out

In [ ]:
res = await gql_all(spaces_query, "spaces")

In [ ]:
res